# Practice #2: Training Convolutional Neural Networks

Now, you're going to leave behind your implementations and instead migrate to one of popular deep learning frameworks, **TensorFlow**. <br>
In this notebook, you will learn how to train convolutional neural networks (CNNs) for classifying images in the CIFAR-10 dataset. <br>
There are **3 sections**, and in each section, you need to follow the instructions to complete the skeleton codes and explain them.

1. [Training a simple CNN model](#1) ( 10 points )
2. [Training a CNN model with Inception modules](#2) ( 10 points )
3. [Design a better model on CIFAR-10](#3) ( 20 points )  

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Some helpful tutorials and references for practice #2:
- [1] TensorFlow official tutorials. [[link]](https://www.tensorflow.org/get_started/get_started)
- [2] Stanford CS231n lectures. [[link]](http://cs231n.stanford.edu/)
- [3] Iandola et al, "SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and < 0.5MB model size", arXiv 2016. [[pdf]](https://arxiv.org/pdf/1602.07360.pdf)
- [4] Simonyan et al., "Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps", ICLR Workshop 2014. [[pdf]](https://arxiv.org/pdf/1312.6034.pdf)
- [5] Szegedy et al., "Intriguing properties of neural networks", ICLR 2014. [[pdf]](https://arxiv.org/pdf/1312.6199.pdf)
- [6] Szegedy et al., "Going deeper with convolutions", CVPR 2015. [[pdf]](http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf)
- [7] Yosinski et al., "Understanding Neural Networks Through Deep Visualization", ICML Workshop 2015. [[pdf]](http://yosinski.com/media/papers/Yosinski__2015__ICML_DL__Understanding_Neural_Networks_Through_Deep_Visualization__.pdf)

## Load datasets
The CIFAR-10 dataset will be downloaded automatically if it is not located in the *Utils* directory. <br>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from Utils.data_utils import load_CIFAR10, plot_images
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
    
conf = tf.ConfigProto()
# conf.gpu_options.per_process_gpu_memory_fraction = 0.2
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
X_train, Y_train, X_val, Y_val, X_test, Y_test, Class_names = load_CIFAR10()
print('Train data shape ' + str(X_train.shape))
print('Train labels shape ' + str(Y_train.shape))
print('Validation data shape ' + str(X_val.shape))
print('Validataion labels shape ' + str(Y_val.shape))
print('Test data shape ' + str(X_test.shape))
print('Test labels shape ' + str(Y_test.shape))
plot_images(X_train, Y_train, Class_names, Each_Category=True)

Data has already been downloaded and unpacked.


FileNotFoundError: [Errno 2] No such file or directory: './Utils/cifar-10-bathes-py/batches.meta'

## <a name="1"></a> 1. Training a basic CNN model ( 10 points )

In this section, you will learn how to define a simple CNN model architecture, train the model, and evaluate its performance on the validation dataset. The goal here isn't to get good performance (that'll be next), but instead to get comfortable with understanding the TensorFlow and configuring a CNN model.

Using the code provided as guidance, **(1) define**, **(2) train**, and **(3) evaluate** a model with the following architecture and training setup:

#### CNN architecture:
* 9x9 Convolutional layer with 8 filters, strides of 1, and ReLU activation
* 3x3 Max pooling layer with strides of 3
* Fully connected layer with 8 output units and ReLU activation
* Fully connected layer with 10 output units and linear activation

#### Training setup:
* Loss function: Sotfmax cross entropy
* Optimizer: Gradient descent with 0.01 learning rate
* Batch size: 32
* Training epoch: 1

In [4]:
# Define our model as a class
# It includes our model architecture, loss function, optimizer, and evaluation metrics
class simple_model(object):
    def __init__(self):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        # Define input variables 
        self.inputs = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.targets = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)

        # Define model architecture
        with tf.variable_scope('L1'):
            L1 = tf.layers.conv2d(inputs=self.inputs, filters=8,\
                kernel_size=[9, 9], strides=[1,1], padding='valid',\
                activation=tf.nn.relu)
            L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[3, 3], \
                strides=[3, 3], padding='valid')
        with tf.variable_scope('L2'):
            L2 = tf.contrib.layers.flatten(L1)
            L2 = tf.layers.dense(L2, 8, activation=tf.nn.relu)
        with tf.variable_scope('Output'):
            self.outputs = tf.layers.dense(L2, 10, activation=None)
        
        # Define loss function and optimizer
        self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=self.outputs, 
                labels=tf.one_hot(self.targets,10)))
        self.optimizer = tf.train.
        GradientDescentOptimizer(0.01).minimize(self.loss)
        
        # Variables for evaluation
        self.corr = tf.equal(
            self.targets, tf.argmax(self.outputs,1))
        self.accuracy = tf.reduce_mean(
            tf.cast(self.corr, tf.float32))   
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################

# Create a function to train and evaluata a model
# You can reuse this function throughout the assignment
def run_model(session, model, X, Y, epochs=5, batch_size=32, is_training=False):
    # For training the model
    if is_training:
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        for epoch in range(epochs):
            total_loss = 0
            # Iterate over the entire dataset
            for i in range(int(X.shape[0] / batch_size)):
                _, cost_val = sess.run([model.optimizer, model.loss], 
                                       feed_dict={model.inputs: X[i*batch_size:(i+1)*batch_size],
                                                  model.targets: Y[i*batch_size:(i+1)*batch_size],
                                                  model.is_training: is_training})
                total_loss += cost_val
            print("Epoch: %02d \t Average loss: %.4f" % (epoch+1, total_loss / X.shape[0] * batch_size))
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################
        print("Training done!")

    # Evaluate loss and accuracy of the model
    else:
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        loss, accuracy = 0.0, 0.0
        for i in range(int(X.shape[0] / batch_size)):
            l, a = sess.run([model.loss, model.accuracy],
                            feed_dict={model.inputs: X[i*batch_size:(i+1)*batch_size],
                                       model.targets: Y[i*batch_size:(i+1)*batch_size],
                                       model.is_training: is_training})
            loss += l
            accuracy += a
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################
        return (loss / X.shape[0] * batch_size, accuracy / X.shape[0] * batch_size)

# Clear old variables
tf.reset_default_graph()    

# Declare out simple model
model = simple_model()    
    
# Now, train and evaluate the model
with tf.Session(config=conf) as sess:
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    sess.run(tf.global_variables_initializer())
    run_model(sess, model, X_train, Y_train, epochs=1, is_training=True)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    print("(Loss, Accuracy) on Training Dataset (%.4f, %.2f)" % run_model(sess, model, X_train, Y_train))
    print("(Loss, Accuracy) on Validataion Dataset (%.4f, %.2f)" % run_model(sess, model, X_val, Y_val))

Epoch: 01 	 Average loss: 2.0041
Training done!
(Loss, Accuracy) on Training Dataset (1.7322, 0.37)
(Loss, Accuracy) on Validataion Dataset (1.7418, 0.36)


## <a name="2"></a>2. Training a CNN model with Inception module ( 10 points )

In this section, you will implement a little more complex CNN model with an `Inception module` [6]. <br>

![Inception_module](Utils/InceptionModule_GoogLeNet.png)

Using the code provided as guidance, <br>
**(1)** Define an `inception module`, which is the building block of **Inception model (a.k.a GoogLeNet)**, the winner of ILSVRC14. <br>
**(2)** Define, train, and evaluate a CNN model with the following architecture and training setup:

#### CNN architecture:
* `Inception module` with 
    * 8 filters for the main convolutions (blue blocks in the Figure(a))
    * 2 filters for the dimensionality reduction convolutions (yellow blocks in the Figure(a))
    *  ReLU activation
* Fully connected layer with 10 output units and linear activation

#### Training setup:
* Loss function: Sotfmax cross entropy
* Optimizer: Gradient descent with 0.01 learning rate
* Batch size: 32
* Training epoch: 5

In [8]:
def Inception_module(Input, C1, C3_R, C3, C5_R, C5, P3_R):
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    conv1 = tf.layers.conv2d(Input, C1, [1, 1], [1, 1], padding='same', activation=tf.nn.relu)
    conv3_R = tf.layers.conv2d(Input, C3_R, [1, 1], [1, 1], padding='same', activation=tf.nn.relu)
    conv3 = tf.layers.conv2d(conv3_R, C3, [3, 3], [1, 1], padding='same', activation=tf.nn.relu)
    conv5_R = tf.layers.conv2d(Input, C5_R, [1, 1], [1, 1], padding='same', activation=tf.nn.relu)
    conv5 = tf.layers.conv2d(conv5_R, C5, [5, 5], [1, 1], padding='same', activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(Input, [3, 3], [1, 1], padding='same')
    pool3_R = tf.layers.conv2d(pool3, P3_R, [1, 1], [1, 1], padding='same', activation=tf.nn.relu)
    Inception = tf.concat([conv1, conv3, conv5, pool3_R], axis=3)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    return Inception

class complex_model(object):
    def __init__(self):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        self.inputs = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.targets = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)

        with tf.variable_scope('L1'):
            L1 = Inception_module(self.inputs, 8, 2, 8, 2, 8, 2)      
        
        with tf.variable_scope('Output'):
            F = tf.contrib.layers.flatten(L1) 
            self.outputs = tf.layers.dense(F, 10, activation=None)
        
        # Define loss function and optimizer
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.outputs, labels=tf.one_hot(self.targets,10)))
        self.optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(self.loss)
        
        # Variables for evaluation
        self.corr = tf.equal(self.targets, tf.argmax(self.outputs,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.corr, tf.float32))   
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ############################################################################## 

# Clear old variables
tf.reset_default_graph() 

# Declare out simple model
model = complex_model()

# Now, create a tf.Session and train the model
with tf.Session(config=conf) as sess:
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    sess.run(tf.global_variables_initializer())
    run_model(sess, model, X_train, Y_train, epochs=5, is_training=True)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    print("(Loss, Accuracy) on Training Dataset (%.4f, %.2f)" % run_model(sess, model, X_train, Y_train))
    print("(Loss, Accuracy) on Validataion Dataset (%.4f, %.2f)" % run_model(sess, model, X_val, Y_val))

Epoch: 01 	 Average loss: 1.6239
Epoch: 02 	 Average loss: 1.3630
Epoch: 03 	 Average loss: 1.2226
Epoch: 04 	 Average loss: 1.1229
Epoch: 05 	 Average loss: 1.0383
Training done!
(Loss, Accuracy) on Training Dataset (0.9519, 0.67)
(Loss, Accuracy) on Validataion Dataset (1.2408, 0.57)


## <a name="3"></a>3. Design a better model on CIFAR-10

Now it's your job to experiment with CNNs to train a model that achieves **<font color=red>>= 70% accuracy on the validation set</font>** of CIFAR-10. <br> You can reuse the implemented functions from above.

### Things you can try to change:
- Filter size
- Number of filters
- Pooling vs Strided Convolution
- Network architectures
- Optimizers
- Activation functions
- Regularizations
- Model ensembles
- Data augmentation

In [5]:
class my_model(object):
    def __init__(self):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        self.inputs = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.targets = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)

        with tf.variable_scope('L1'):
            L1 = Inception_module(self.inputs, 32, 4, 32, 4, 32, 4)      
            L1 = tf.layers.max_pooling2d(L1, pool_size=[3, 3], strides=[2, 2], padding='valid')
            L1 = tf.layers.dropout(L1, 0.7, self.is_training)
            
        with tf.variable_scope('L2'):
            L2 = Inception_module(L1, 64, 8, 64, 8, 64, 8)      
            L2 = tf.layers.max_pooling2d(L2, pool_size=[3, 3], strides=[2, 2], padding='valid')
            L2 = tf.layers.dropout(L2, 0.7, self.is_training)
        
        with tf.variable_scope('L3'):
            L3 = tf.contrib.layers.flatten(L2)
            L3 = tf.layers.dense(L3, 64, activation=tf.nn.relu)
            L3 = tf.layers.dropout(L3, 0.7, self.is_training)
        
        with tf.variable_scope('Output'):
            self.outputs = tf.layers.dense(L3, 10, activation=None)
        
        # Define loss function and optimizer
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.outputs, labels=tf.one_hot(self.targets,10)))
        self.optimizer = tf.train.AdamOptimizer().minimize(self.loss)
        
        # Variables for evaluation
        self.corr = tf.equal(self.targets, tf.argmax(self.outputs,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.corr, tf.float32))   
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################

# Clear old variables
tf.reset_default_graph()    

# Declare out simple model
model = my_model()    
    
# Now, create a tf.Session and train the model
with tf.Session(config=conf) as sess:
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    sess.run(tf.global_variables_initializer())
    run_model(sess, model, X_train, Y_train, epochs=10, batch_size=100, is_training=True)
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################
    print("(Loss, Accuracy) on Training Dataset (%.4f, %.2f)" % run_model(sess, model, X_train, Y_train))
    print("(Loss, Accuracy) on Validataion Dataset (%.4f, %.2f)" % run_model(sess, model, X_val, Y_val))
    
    #Save your final model
    saver = tf.train.Saver()
    saver.save(sess, "./Utils/model_checkpoints/my_model_final")

Epoch: 01 	 Average loss: 1.4107
Epoch: 02 	 Average loss: 0.9871
Epoch: 03 	 Average loss: 0.8227
Epoch: 04 	 Average loss: 0.7079
Epoch: 05 	 Average loss: 0.6190
Epoch: 06 	 Average loss: 0.5490
Epoch: 07 	 Average loss: 0.4986
Epoch: 08 	 Average loss: 0.4327
Epoch: 09 	 Average loss: 0.3837
Epoch: 10 	 Average loss: 0.3574
Training done!
(Loss, Accuracy) on Training Dataset (0.3956, 0.86)
(Loss, Accuracy) on Validataion Dataset (1.0951, 0.70)


### Describe what you did here
In this cell you should also write an explanation of what you did, any additional features that you implemented, and any visualizations or graphs that you make in the process of training and evaluating your network.

_Tell us here_

### Test Set - Do this only once
Now that you've gotten a result that you're happy with, test your final model on the test set. This would be the score you would achieve on a competition. Think about how this compares to your validation set accuracy.

In [6]:
# Clear old variables
tf.reset_default_graph()  

with tf.Session(config=conf) as sess:
    #Load your final model
    model = my_model()
    saver = tf.train.Saver()
    saver.restore(sess, "./Utils/model_checkpoints/my_model_final")
    print("(Loss, Accuracy) on Test Dataset (%.4f, %.2f)" % run_model(sess, model, X_test, Y_test))

INFO:tensorflow:Restoring parameters from ./Utils/model_checkpoints/my_model_final
(Loss, Accuracy) on Test Dataset (1.0912, 0.70)
